In [ ]:
.libPaths("/home/mahat/.conda/envs/r422/lib/R/library")
.libPaths()

In [ ]:
suppressMessages({
    library(tidyverse)
    library(ggExtra)
    library(matrixStats)
    library(plyranges)
    library(viridis)
    library(data.table)
    library(ggbio)
    library(ggridges)
    library(ggbeeswarm)
    library(ggpointdensity)
    library(Matrix)
});
source("./scGRO_functions.r");

In [ ]:
options(
    repr.plot.width=6,
    repr.plot.height=4,
    jupyter.plot_mimetypes = "image/svg+xml"
);
theme_set(theme_classic() +
    theme(
        axis.title.x = element_text(color="black", size=14, face="bold"), 
        axis.title.y = element_text(color="black", size=14, face="bold"),
        axis.text = element_text(color="black", size=12, face="bold"),
        plot.title = element_text(face="bold", size=14, hjust = 0.5),
        axis.line = element_blank(),
        # axis.ticks = element_blank()
        panel.border = element_rect(colour = "grey", fill=NA, linewidth=1)
    )
);

In [ ]:
# Get equation and r^2 as string
# https://groups.google.com/forum/#!topic/ggplot2/1TgH-kG5XMA

lm_eqn = function(x, y) {
    m = lm(y ~ x);
    eq = substitute(italic(y) == a + b %.% italic(x)*","~~italic(r)^2~"="~r2, 
         list(a = format(unname(coef(m)[1]), digits = 2),
              b = format(unname(coef(m)[2]), digits = 2),
             r2 = format(summary(m)$r.squared, digits = 3)))
    as.character(as.expression(eq));
}

In [ ]:
#features = read_bed("../data/groHMM_mES_BRsComb_LP-50_UTS10_features_customized_v2.bed");
# features = read_bed("../data/mES_BRsComb_dREGfiltered_features_customized_OSNenhancersPlusSEs_v1.bed");
features = read_bed("../data/dREG_refinedFeatures_mES_mm10_OSNcustomEnhancers_SEs.bed");
names(features) = features$name;
features$score=NULL;
features

# # select genes longer than 5kp from each feature
# # truncate genes longer than 30kb to 30kb
# features = features %>%
#     filter( width(features) >= 3000 | substr(name, 0, 2) != "GN" ) %>%
#     anchor_center() %>%
#     mutate(width=width-1000);
# length(features)

#resize 30+kb genes to 30kb
longf = which( width(features) > 30000 & substr(names(features), 0, 2) == "GN" );
features[longf] = features[longf] %>%
     resize( width = 30000, fix="start" );
features

In [ ]:
scGRO  = readRDS("../data/scGROv2p8_consolidated.rds");
counts = readRDS( "../data/scGROv2p8_mapq3qc_filtered_counts.rds" );
# scGRO  = readRDS("../jay_m/data_jay_m/scGROv2p9_filtered_consolidated.rds");
# counts = readRDS( "../jay_m/data_jay_m/scGROv2p9_mapq3qc_filtered_counts.rds" );
# hist(colSums(counts))
# head(scGRO)
# head(counts)

In [ ]:
# Merge experiment ID and cell barcode to create unique cell ID across experiments
scGRO = scGRO %>%
    filter( umiQC & plateQC & cellQC & countQC & miRQC ) %>%
    mutate(cellID = factor( paste( Exp, Plate, Cell, sep="-") ) ) %>%
    # filter( !(Exp == "Exp236" & Plate %in% paste0("c0", 5:8)) ) %>%
    # filter( !(Exp == "Exp260b") ) %>%
    resize(width=1, fix="end") %>%
    filter(cellID %in% colnames(counts)) %>%
    # subsetByOverlaps(features) %>%
    select( cellID );
names(scGRO) = NULL;
scGRO

In [ ]:
scrambled = scGRO;
scrambled$cellID = droplevels(scrambled$cellID);
scrambled$cellID = sample(scrambled$cellID);

In [ ]:
# load dREG peak calls and convert to GRanges
dREG = read.table("../data/PROseq_mES_BRsComb.dREG.peak.full.bed", header=F, stringsAsFactors=F);
colnames(dREG) = c("chr", "start", "end", "score", "pval", "center");
dREG = GRanges(dREG)
dREG

### plot distance between Pol II in gene and enhancer in same cells and thier positions

In [ ]:
geneA = features["GN-Hccs"]
geneB = features["GN-LOC101055907"]
# strand(geneA) = "+";
# strand(geneB) = "-"
plot_position_pairs( scGRO, geneA, geneB, dREG );
plot_position_table( scGRO, geneA, geneB );

In [ ]:
gene = features["GN-Sox2"]
enh = features["Sox2_2kbDn"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Sox2"]
enh = features["Sox2_105kbDn"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Sox2"]
enh = features["Sox2_1MbDn"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Pou5f1"]
enh = features["Pou5f1_3kbUp"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Pou5f1"]
enh = features["Pou5f1_25kbUp"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Pou5f1"]
enh = features["Pou5f1_20kbUp"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Sulf1"]
enh = features["INT_STITCHED_45"]
# plot_2strand_position_pairs(scGRO, gene, enh, dREG);
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Sulf1"]
enh = features["INT_STITCHED_45"]
start(gene) = 12692262;
gene = resize(gene, width=30000, fix="start");
start(enh) = 13069651;
strand(enh) = "+";
enh = resize(enh, width=30000, fix="start");
plot_position_pairs( scGRO, gene, enh, dREG );
end(enh) = 13095010;
strand(enh) = "-";
enh = resize(enh, width=30000, fix="start");
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Slco5a1"]
enh = features["INT_STITCHED_45"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Nr5a2"]
enh = features["INT_STITCHED_466"]
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Etl4"]
enh = features["INT_STITCHED_746"];
strand(enh) = "+";
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Dnmt3b"]
enh = features["INT_STITCHED_1210"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Cbfa2t2"]
enh = features["INT_STITCHED_1210"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );	

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gar1"]
enh = features["INT_STITCHED_1732"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );	

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rad23b"]
enh = features["INT_STITCHED_1973"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rcc2"]
enh = features["INT_STITCHED_2292"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Kazn"]
enh = features["INT_STITCHED_2292"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Med13l"]
enh = features["INT_STITCHED_2745"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"	
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Pml"]
enh = features["INT_STITCHED_4555"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gclc"]
enh = features["INT_STITCHED_4657"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ooep"]
enh = features["INT_STITCHED_4657"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gtf3c6"]
enh = features["INT_STITCHED_4954"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Herc4"]
enh = features["INT_STITCHED_5044"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Hnrnph3"]
enh = features["INT_STITCHED_5044"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Epn2"]
enh = features["INT_STITCHED_5533"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-LOC108167924"]
enh = features["INT_STITCHED_5711"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ddx24"]
enh = features["INT_STITCHED_6151"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ptch1"]
enh = features["INT_STITCHED_6460"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ctsl"]
enh = features["INT_STITCHED_6460"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Basp1"]
enh = features["INT_STITCHED_7104"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Myo10"]
enh = features["INT_STITCHED_7104_2"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Usp7"]
enh = features["INT_STITCHED_7359"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-H2-M5"]
enh = features["INT_STITCHED_7792"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Wdr43"]
enh = features["INT_STITCHED_7884"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Myl12a"]
enh = features["INT_STITCHED_7884"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Nr5a2"]
enh = features["INT_STITCHED_466"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Aldh9a1"]
enh = features["INT_STITCHED_559"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Sde2"]
enh = features["INT_STITCHED_611"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Etl4"]
enh = features["INT_STITCHED_746"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gpr107"]
enh = features["INT_STITCHED_803"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Csnk2a1"]
enh = features["INT_STITCHED_1196"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Cbfa2t2"]
enh = features["INT_STITCHED_1210"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gm39962"]
enh = features["INT_STITCHED_1210"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gm14261"]
enh = features["INT_STITCHED_1300"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Trim33"]
enh = features["INT_STITCHED_1658"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Slc37a3"]
enh = features["INT_STITCHED_3045"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Nanog"]
enh = features["INT_STITCHED_3348"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Etnk1"]
enh = features["INT_STITCHED_3437"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Pml"]
enh = features["INT_STITCHED_4555"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ooep"]
enh = features["INT_STITCHED_4657"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rnu3a"]
enh = features["INT_STITCHED_4954"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-2410006H16Rik"]
enh = features["INT_STITCHED_5533"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-E130012A19Rik"]
enh = features["INT_STITCHED_5711"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-LOC108167924"]
enh = features["INT_STITCHED_5711"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rara"]
enh = features["INT_STITCHED_5719"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Mapt"]
enh = features["INT_STITCHED_5752"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ppp2r5c"]
enh = features["INT_STITCHED_6188_2"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-LOC102638940"]
enh = features["INT_STITCHED_6188_2"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ppp2r5c"]
enh = features["INT_STITCHED_6188"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gm30363"]
enh = features["INT_STITCHED_6709"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Dmtn"]
enh = features["INT_STITCHED_6887"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Tsc22d1"]
enh = features["INT_STITCHED_6904"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Myo10"]
enh = features["INT_STITCHED_7104_2"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Usp7"]
enh = features["INT_STITCHED_7359"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-H2-M5"]
enh = features["INT_STITCHED_7792"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rab12"]
enh = features["INT_STITCHED_7876_2"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gm36856"]
enh = features["INT_STITCHED_7887"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Gm20544"]
enh = features["INT_STITCHED_8261"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Dst"]
enh = features["INT_STITCHED_88"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Nr5a2"]
enh = features["INT_STITCHED_466"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Uck2"]
enh = features["INT_STITCHED_556"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Etl4"]
enh = features["INT_STITCHED_746"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Cbfa2t2"]
enh = features["INT_STITCHED_1210"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Zfp704"]
enh = features["INT_STITCHED_1392"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-B330016D10Rik"]
enh = features["INT_STITCHED_2291"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Kazn"]
enh = features["INT_STITCHED_2292"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Med13l"]
enh = features["INT_STITCHED_2745"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Nanog"]
enh = features["INT_STITCHED_3348"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Phc1"]
enh = features["INT_STITCHED_3348"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Idh2"]
enh = features["INT_STITCHED_3652"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Tead1"]
enh = features["INT_STITCHED_3765"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Utf1"]
enh = features["INT_STITCHED_3890"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Chd9"]
enh = features["INT_STITCHED_4179"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Hmg20a"]
enh = features["INT_STITCHED_4555"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);	

In [ ]:
gene = features["GN-Dppa5a"]
enh = features["INT_STITCHED_4657"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Lrrc2"]
enh = features["INT_STITCHED_4748"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Derl3"]
enh = features["INT_STITCHED_5091"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rara"]
enh = features["INT_STITCHED_5719"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Mapt"]
enh = features["INT_STITCHED_5752"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Esrrb"]
enh = features["INT_STITCHED_6113"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Ppp2r5c"]
enh = features["INT_STITCHED_6188"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Tsc22d1"]
enh = features["INT_STITCHED_6904"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Tsc22d1"]
enh = features["INT_STITCHED_6906"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Rpap3"]
enh = features["INT_STITCHED_7317"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Usp7"]
enh = features["INT_STITCHED_7359"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-H2-M5"]
enh = features["INT_STITCHED_7792"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
gene = features["GN-Kctd16"]
enh = features["INT_STITCHED_8140"];
strand(enh) = "+"
plot_position_pairs( scGRO, gene, enh, dREG );
strand(enh) = "-"
plot_position_pairs( scGRO, gene, enh, dREG );

plot_position_table(scGRO, gene, enh);

In [ ]:
marginal_corr = function( reads, gene, enh ) {
    query = features[ c(gene, enh) ];
    qstart= promoters(query, upstream=0, downstream=1);
    
    genepol = subsetByOverlaps(reads, query[1]) %>%
        mutate( gene = distanceToNearest(., qstart[1]) );
    genepol$gstrand = as.character(strand(genepol));
    
    enhpol = subsetByOverlaps(reads, query[2]) %>%
        mutate( enh = distanceToNearest(., qstart[2]) );
    enhpol$estrand = as.character(strand(enhpol));

    genepol = mcols(genepol) %>%
        as.data.frame %>%
        filter(cellID %in% enhpol$cellID) %>%
        select(cellID, gene.distance, gstrand);
    
    enhpol  = mcols(enhpol) %>%
        as.data.frame %>%
        filter(cellID %in% genepol$cellID) %>%
        select(cellID, enh.distance, estrand);

    p = left_join(genepol, enhpol, by="cellID") %>%
            ggplot(aes(x=gene.distance/1000, y=enh.distance/1000)) +
            geom_point() +
            xlim(0, 50) +
            ylim(0, 20) +
            xlab("Gene distance transcribed (kb)") +
            ylab("Enhancer distance transcribed (kb)") +
            theme(legend.position="none");

    ggMarginal(p, type="histogram")
}

In [ ]:
marginal_corr( scGRO, "GN-Sox2", "Sox2_105kbDn_pl" );

In [ ]:
marginal_corr( scGRO, "GN-Pou5f1", "Pou5f1_25kbUp_mn" );